# Studying COVID-19

### Setup

Let's setup Spark Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.4 MB 64 kB/s 
     |████████████████████████████████| 198 kB 66.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=1dea60acc65211ab05e80e2f01ec83e07c4d6382bada79d8a171969d1bb30c46
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Rea

Authenticating a Google Drive client to download the files we will be process on Spark job.

In [2]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1YT7ttUAafCjbVdm6obeHp1TWAK0rEtoR'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_covid19_confirmed_global.csv')

id='1YxEA5UQ2EFJ_9oLssM__Gs1ncVNufGNA'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_covid19_deaths_global.csv')

id='1CNxszuZTeIw-5cF5yrzKMZdb1qV0hSoy'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_covid19_recovered_global.csv')

If you executed the cells above, you should be able to see the dataset I will be using for this Colab under the "Files" tab on the left panel.

Next, Let's import some of the common libraries needed for our task.

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [15]:
sc.stop()

In [16]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [17]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [18]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-10-06 23:23:04--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 52.202.168.65, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.19M  5.68MB/s    in 2.3s    

2021-10-06 23:23:07 (5.68 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes
  inflating: ngrok                   
https://1853-34-132-73-148.ngrok.io


### Data Loading

In this Notebook, I will be analyzing the timeseries data of the Coronavirus COVID-19 Global Cases, collected by Johns Hopkins CSSE.

Here you can check a realtime dashboard based on this dataset: [https://www.arcgis.com/apps/opsdashboard/index.html?fbclid=IwAR2hQKsEZ3D38wVtXGryUhP9CG0Z6MYbUM_boPEaV8FBe71wUvDPc65ZG78#/bda7594740fd40299423467b48e9ecf6](https://www.arcgis.com/apps/opsdashboard/index.html?fbclid=IwAR2hQKsEZ3D38wVtXGryUhP9CG0Z6MYbUM_boPEaV8FBe71wUvDPc65ZG78#/bda7594740fd40299423467b48e9ecf6)

---



*   ```confirmed```: dataframe containing the cumulative number of confirmed COVID-19 cases, divided by geographical area
*   ```deaths```: dataframe containing the cumulative number of deaths due to COVID-19, divided by geographical area
*   ```recovered```: dataframe containing the cumulative number of recoevered patients, divided by geographical area

The data sets contain data entries for each day, representing the cumulative totals as of that day.







In [19]:
confirmed = spark.read.csv('time_series_covid19_confirmed_global.csv', header=True)
deaths = spark.read.csv('time_series_covid19_deaths_global.csv', header=True)
recovered = spark.read.csv('time_series_covid19_recovered_global.csv', header=True)

In [20]:
confirmed.printSchema()

root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: string (nullable = true)
 |-- Long: string (nullable = true)
 |-- 1/22/20: string (nullable = true)
 |-- 1/23/20: string (nullable = true)
 |-- 1/24/20: string (nullable = true)
 |-- 1/25/20: string (nullable = true)
 |-- 1/26/20: string (nullable = true)
 |-- 1/27/20: string (nullable = true)
 |-- 1/28/20: string (nullable = true)
 |-- 1/29/20: string (nullable = true)
 |-- 1/30/20: string (nullable = true)
 |-- 1/31/20: string (nullable = true)
 |-- 2/1/20: string (nullable = true)
 |-- 2/2/20: string (nullable = true)
 |-- 2/3/20: string (nullable = true)
 |-- 2/4/20: string (nullable = true)
 |-- 2/5/20: string (nullable = true)
 |-- 2/6/20: string (nullable = true)
 |-- 2/7/20: string (nullable = true)
 |-- 2/8/20: string (nullable = true)
 |-- 2/9/20: string (nullable = true)
 |-- 2/10/20: string (nullable = true)
 |-- 2/11/20: string (nullable = true)
 |-- 2/12/20: string (

In [21]:
def md5(s):
  import hashlib
  m = hashlib.md5()
  m.update(s.encode())
  return m.hexdigest()[:2]

md5('3/5/20')
confirmed.show()

+--------------------+-------------------+---------+----------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-----

### Analyzing COVID-19 timeseries data

Consider the entries for May 1, 2021, in the timeseries, and compute:


*   number of confirmed COVID-19 cases across the globe
*   number of deaths due to COVID-19 (across the globe)
*   number of recovered patients across the globe



In [22]:
def list_sum(l):
  sum = 0
  for x in l:
    sum += int(x)
  return sum
confirmed_list = confirmed.select('3/5/20').rdd.flatMap(lambda x: x).collect()
deaths_list = deaths.select('3/5/20').rdd.flatMap(lambda x: x).collect()
recovered_list = recovered.select('3/5/20').rdd.flatMap(lambda x: x).collect()

print("number of confirmed COVID-19 cases across the globe is: {}".format(list_sum(confirmed_list)))
print("number of deaths due to COVID-19 cases across the globe is: {}".format(list_sum(deaths_list)))
print("number of recovered patients across the globe is: {}".format(list_sum(recovered_list)))

number of confirmed COVID-19 cases across the globe is: 98090
number of deaths due to COVID-19 cases across the globe is: 3349
number of recovered patients across the globe is: 53795


Consider only the most recent data points for March 1, 2020, and March 1, 2021 in the timeseries, and filter out the geographical locations where less than 50 cases have been confirmed. For the areas still taken into consideration after the filtering step, compute the ratio between number of recovered patients and number of confirmed cases.

In [24]:
confirmed_list = confirmed.select('Province/State', 'Country/Region', col('3/5/20').alias('confirmed'))
deaths_list = deaths.select('Province/State', 'Country/Region', col('3/5/20').alias('deaths'))
recovered_list = recovered.select('Province/State', 'Country/Region', col('3/5/20').alias('recovered'))
result = confirmed_list.join(deaths_list, ['Province/State', 'Country/Region'])
result = result.join(recovered_list, ['Province/State', 'Country/Region'])
result = result.withColumn("%recovered", result.recovered / result.confirmed)
result = result.withColumn("%deaths", result.deaths / result.confirmed)


Following the same filtering strategy as above, I am now computing the ratio between number of deaths and number of confirmed cases.

In [25]:
result = result.filter(result.confirmed >= 50).sort("%recovered", ascending = False)
result.show()

+--------------+--------------+---------+------+---------+------------------+--------------------+
|Province/State|Country/Region|confirmed|deaths|recovered|        %recovered|             %deaths|
+--------------+--------------+---------+------+---------+------------------+--------------------+
|         Anhui|         China|      990|     6|      970|0.9797979797979798|0.006060606060606061|
|         Henan|         China|     1272|    22|     1239|0.9740566037735849| 0.01729559748427673|
|        Yunnan|         China|      174|     2|      169|0.9712643678160919|0.011494252873563218|
|       Jiangxi|         China|      935|     1|      901|0.9636363636363636|0.001069518716577...|
|         Hebei|         China|      318|     6|      304|0.9559748427672956|0.018867924528301886|
|        Shanxi|         China|      133|     0|      126|0.9473684210526315|                 0.0|
|         Jilin|         China|       93|     1|       88| 0.946236559139785|0.010752688172043012|
|       Ti

In [27]:
result = result.filter(result.confirmed >= 50).sort("%deaths", ascending = False)
result.show()

+--------------+--------------+---------+------+---------+------------------+--------------------+
|Province/State|Country/Region|confirmed|deaths|recovered|        %recovered|             %deaths|
+--------------+--------------+---------+------+---------+------------------+--------------------+
|         Hubei|         China|    67466|  2902|    40592|0.6016660243678297| 0.04301425903418018|
|      Xinjiang|         China|       76|     3|       70|0.9210526315789473|0.039473684210526314|
|        Hainan|         China|      168|     6|      158|0.9404761904761905| 0.03571428571428571|
|  Heilongjiang|         China|      481|    13|      379|0.7879417879417879| 0.02702702702702703|
|       Tianjin|         China|      136|     3|      128|0.9411764705882353|0.022058823529411766|
|         Gansu|         China|      102|     2|       87|0.8529411764705882|  0.0196078431372549|
|       Beijing|         China|      418|     8|      297|0.7105263157894737|0.019138755980861243|
|     Hong